In [25]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [26]:
!cp -R /content/drive/MyDrive/COSRMAL_CHALLENGE/train /content/train

^C


In [27]:
!cp -R /content/drive/MyDrive/COSRMAL_CHALLENGE/pyAudioAnalysis /content/pyAudioAnalysis

^C


In [ ]:
!pip install eyed3
!pip install pydub

In [ ]:
!wget https://corsmal.eecs.qmul.ac.uk/data/CCM/ccm_train_annotation.json

In [37]:
import json
import pandas as pd
import os
import numpy as np
from scipy.io import wavfile
import matplotlib.pyplot as plt
import IPython
import scipy
import wave
import seaborn as sns
import IPython.display as ipd
import librosa
import torch
import random
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn import Module
from pyAudioAnalysis import audioTrainTest as aT
from pyAudioAnalysis import MidTermFeatures as aF

In [44]:
f = open('./ccm_train_annotation.json')
data = json.load(f)

gt = pd.DataFrame(data['annotations'])
assert len(gt) == len(os.listdir('./train/audio')), "Same number of data"

gt['nframes'] = gt['id'].apply(lambda f: wave.open('./train/audio/' + '{:06d}'.format(f)+'.wav').getnframes())
gt.head()


,id,container id,scenario,background,illumination,width at the top,width at the bottom,height,depth,container capacity,container mass,filling type,filling level,filling density,filling mass,object mass,handover starting frame,handover start timestamp,handover hand,nframes
0,0,2,2,1,0,69.0,42.0,72.0,-1.0,185.000,2.0,2,1,0.82,76.0,78.0,-1,-1,-1,291576
1,1,7,0,0,0,193.0,193.0,241.0,69.0,3209.397,59.0,0,0,0.00,0.0,59.0,-1,-1,-1,118483
2,2,2,0,1,0,69.0,42.0,72.0,-1.0,185.000,2.0,3,1,1.00,93.0,95.0,-1,-1,-1,572008
3,3,8,0,1,0,135.0,135.0,164.0,56.0,1239.840,31.0,0,0,0.00,0.0,31.0,-1,-1,-1,141680
4,4,4,1,1,0,88.0,56.0,91.0,-1.0,296.000,86.0,1,1,0.34,45.0,131.0,-1,-1,-1,138681


In [ ]:
_, ax = plt.subplots(figsize=(16, 4))
sns.violinplot(ax=ax, x="filling type", y="nframes", data=gt)
plt.xticks(rotation=90)
plt.title('Distribution of audio frames, per label', fontsize=16)
plt.show()

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(16,5))
gt.nframes.hist(bins=100, ax=axes)
plt.suptitle('Frame Length Distribution in Train and Test', ha='center', fontsize='large');

In [ ]:
IPython.display.display(ipd.Audio('./train/audio/000377.wav'))

In [ ]:
# ann = pd.read_csv('annotations.csv')
# ann.head()

In [ ]:
input_audio_filename = "./train/audio/000000.wav"
samplerate, data = wavfile.read(input_audio_filename)
print(samplerate, data.shape)

In [ ]:
length = data.shape[0] / samplerate
time = np.linspace(0., length, data.shape[0])

for i in range(data.shape[1]):
  plt.plot(time, data[:, i], label="{} channel".format(i))

# 8 view sound
plt.show()

In [ ]:
data, _ = librosa.core.load('./train/audio/000000.wav', 
                                    sr=160000,
                                    res_type='kaiser_fast')

In [ ]:
IPython.display.Audio(data, rate=samplerate)

In [ ]:

audio_path = os.listdir('./train/audio')
audios = []

longest = 0
shortest = float('inf')
for files in audio_path:
  samplerate, data = wavfile.read(os.path.join('./train/audio', files))

  if data.shape[0] > longest:
    print(files)
    longest = data.shape[0]
    longest_audio = data
  
  if data.shape[0] < shortest:
    shortest = data.shape[0]
    shortest_audio = data

print(longest)
print(shortest)


In [ ]:
IPython.display.Audio(longest_audio[:, 0], rate=samplerate)

In [ ]:
IPython.display.Audio(shortest_audio[:, 0], rate=samplerate)

In [ ]:
data, _ = librosa.core.load('./train/audio/000001.wav', 
                                    sr=44100,
                                    res_type='kaiser_fast')
IPython.display.Audio(data, rate=44100)

In [ ]:
data_mfcc = librosa.feature.mfcc(data, sr=44100, n_mfcc=40)
plt.imshow(data_mfcc, cmap='hot', interpolation='nearest');

In [ ]:
class audioCorsmal(Dataset):
    def __init__(self, data_dir, label, config):
        self.data_dir = [os.path.join(data_dir, path) for path in sorted(os.listdir(data_dir))]
        self.label = label

        self.config = config

    def __len__(self):
        return len(self.data_dir)
    
    def __getitem__(self, idx):
        data, _ = librosa.core.load(self.data_dir[idx], 
                                    sr=self.config.sampling_rate,
                                    res_type='kaiser_fast')
        
        
            





In [58]:
from pyAudioAnalysis import audioBasicIO as aIO 


def midTermFeature_extraction(file_path):
  sampling_rate, signal = aIO.read_audio_file(file_path)
  signal = np.mean(signal,axis=1)
  mid_features, short_features, mid_feature_names = \
                  aF.mid_feature_extraction(signal, sampling_rate,
                                         round(1.0 * sampling_rate),
                                         round(1.0 * sampling_rate),
                                         round(sampling_rate * aT.shortTermWindow),
                                         round(sampling_rate * aT.shortTermStep))
                  
  mid_features = np.transpose(mid_features)
  mid_features = mid_features.mean(axis=0)

  return mid_features


-24.832583874746444


In [56]:
print(gt[gt['filling type']==3]['id'].shape)


(144,)


In [70]:
from tqdm.notebook import tqdm
audio_folder = './train/audio'

audio_paths = [os.path.join(audio_folder, path) for path in sorted(os.listdir(audio_folder))]
audio_features = np.array([midTermFeature_extraction(audio_path) for audio_path in tqdm(audio_paths)])


  0%|          | 0/684 [00:00<?, ?it/s]

In [77]:
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(audio_features, 
                                                    gt['filling type'], 
                                                    test_size=0.2, 
                                                    random_state=42)

In [87]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(train_x)

train_x = scaler.transform(train_x)
test_x = scaler.transform(test_x)

In [93]:
import xgboost as xgb
from sklearn.metrics import accuracy_score

xgboost_model = xgb.XGBClassifier(learning_rate=0.03,
                                  n_estimators=200,
                                  max_depth=5,
                                  base_score=0.6,
                                  gamma=1,
                                  subsample=0.3,
                                  colsample_bytree=0.7,
                                  colsample_bylevel=0.4, 
                                  scale_pos_weight=5) #learning_rate=0.02,n_estimators=200,max_depth=5,base_score=0.5,scale_pos_weight=1
xgboost_model.fit(train_x, train_y)
prediction=xgboost_model.predict(test_x)
print("Acc: ",accuracy_score(test_y,prediction))


Acc:  0.9416058394160584


In [94]:
from numpy import sort
from sklearn.feature_selection import SelectFromModel
thresholds = np.sort(xgboost_model.feature_importances_)


for thresh in thresholds[-120:]:
    selection = SelectFromModel(xgboost_model,threshold = thresh,prefit=True)
    select_X_train = selection.transform(train_x)
    selection_model = xgb.XGBClassifier(learning_rate=0.03,
                                        n_estimators=200,
                                        max_depth=5,
                                        base_score=0.6,
                                        gamma=1,
                                        subsample=0.3,
                                        colsample_bytree=0.7,
                                        colsample_bylevel=0.4, 
                                        scale_pos_weight=5)
    selection_model.fit(select_X_train, train_y.values.ravel())
    select_X_test = selection.transform(test_x)
    prediction = selection_model.predict(select_X_test)
    
    
    print("Thresh=%.3f, n=%d, Acc:%.2f%%" %
          (thresh, select_X_train.shape[1], accuracy_score(test_y,prediction) * 100))
    

Thresh=0.004, n=120, Acc:93.43%
Thresh=0.004, n=119, Acc:94.16%
Thresh=0.004, n=118, Acc:94.89%
Thresh=0.004, n=117, Acc:93.43%
Thresh=0.004, n=116, Acc:93.43%
Thresh=0.004, n=115, Acc:94.16%
Thresh=0.004, n=114, Acc:92.70%
Thresh=0.004, n=113, Acc:92.70%
Thresh=0.004, n=112, Acc:94.16%
Thresh=0.004, n=111, Acc:94.16%
Thresh=0.004, n=110, Acc:92.70%
Thresh=0.004, n=109, Acc:92.70%
Thresh=0.005, n=108, Acc:93.43%
Thresh=0.005, n=107, Acc:92.70%
Thresh=0.005, n=106, Acc:94.16%
Thresh=0.005, n=105, Acc:92.70%
Thresh=0.005, n=104, Acc:92.70%
Thresh=0.005, n=103, Acc:93.43%
Thresh=0.005, n=102, Acc:93.43%
Thresh=0.005, n=101, Acc:91.97%
Thresh=0.005, n=100, Acc:92.70%
Thresh=0.005, n=99, Acc:92.70%
Thresh=0.005, n=98, Acc:92.70%
Thresh=0.005, n=97, Acc:94.16%
Thresh=0.005, n=96, Acc:94.16%
Thresh=0.005, n=95, Acc:92.70%
Thresh=0.005, n=94, Acc:93.43%
Thresh=0.005, n=93, Acc:94.89%
Thresh=0.005, n=92, Acc:94.16%
Thresh=0.005, n=91, Acc:93.43%
Thresh=0.005, n=90, Acc:94.89%
Thresh=0.005, n=89